## Test 1 : XGboost

In [5]:
import xgboost as xgb
from xgboost import plot_importance
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import lightgbm
import catboost

In [1]:
from sklearn import datasets
digits = datasets.load_digits()

In [2]:
images = digits.images
targets = digits.target
print(images.shape)

(1797, 8, 8)


In [3]:
images = images.reshape(1797, 8*8)

In [4]:
x_train, x_valid, y_train, y_valid = train_test_split(images, targets, test_size=0.2)
print(x_train.shape, y_train.shape)

NameError: name 'train_test_split' is not defined

In [ ]:
model = xgb.XGBClassifier(learning_rate=0.1,
    n_estimators=100,
    max_depth=5,
    min_child_weight=1,
    gamma= 0,
    colsample_bylevel= 0.8,
    nthread= 4,
    seed= 27)
model.fit(x_train, y_train)

In [ ]:
y_pred = model.predict(x_valid)

In [ ]:
from xgboost import plot_importance
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
confusion = confusion_matrix(y_valid, y_pred)
print('confusion matrix')
print(confusion)
plot_importance(model)
print(classification_report(y_valid, y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV

param_test = {
    'max_depth': [4, 5, 6],
    'min_child_weight': [4, 5, 6]
}
xgbGS = xgb.XGBClassifier(learning_rate=0.1,
                    n_estimators=100,
                    max_depth=5,
                    min_child_weight=2,
                    gamma= 0,
                    colsample_bylevel= 0.8,
                    nthread= 4,
                    seed= 27)
gsearch = GridSearchCV(estimator=xgbGS, param_grid=param_test, cv=5)
gsearch_model = gsearch.fit(x_train, y_train)

In [ ]:
gsearch_preds = gsearch_model.predict(x_valid)
print(classification_report(y_valid, y_pred))
print(accuracy_score(y_valid, y_pred) * 100)

## Test 2 LightGBM

In [ ]:
from lightgbm import LGBMClassifier

lightgbm_model = LGBMClassifier()
lightgbm_model.fit(x_train, y_train)

In [ ]:
y_pred = lightgbm_model.predict(x_valid)

In [ ]:
print(accuracy_score(y_valid, y_pred) * 100)
lightgbm_model.booster_.save_model('./lightgbm_model.txt')
bst = lightgbm.Booster(model_file="./lightgbm_model.txt")
y_pred = bst.predict(x_valid)
print(y_pred.shape)

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score


cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
n_scores = cross_val_score(lightgbm_model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

print(np.mean(n_scores), np.std(n_scores))

In [ ]:
lightgbm_model.feature_importances_

In [ ]:
param_test = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40]
}

gsearch_lightgbm_model = GridSearchCV(estimator=lightgbm_model, param_grid=param_test, cv=5)
gsearch_lightgbm_model.fit(x_train, y_train)
print('best params: ', gsearch_lightgbm_model.best_params_)

In [ ]:
y_pred = gsearch_lightgbm_model.predict(x_valid)
print(accuracy_score(y_valid, y_pred) * 100)

In [ ]:
gsearch_lightgbm_model.best_estimator_.booster_.save_model('./gsearch_lightgbm_model.txt')

In [ ]:
bst = lightgbm.Booster(model_file="./gsearch_lightgbm_model.txt")